<a href="https://colab.research.google.com/github/Akshatpattiwar512/Big-data-ml/blob/main/Apache_Spark_Part_2_Spark_SQL_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Apache Spark-Part 2: Spark SQL/DataFrames


Following notebook is based on [Apache Spark-Part 2: Spark SQL/DataFrames](https://www.linkedin.com/pulse/apache-spark-part-2-spark-sqldataframes-akshat-pattiwar/) article

DataFrames are logically identical to relational tables or DataFrames in Python/R, but they have a lot of optimizations hidden behind the scenes. We can make DataFrames from collections, HIVE tables, Relational tables, and RDDs in a variety of methods.

##Installing pySpark

In [10]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 63kB/s 
     |████████████████████████████████| 204kB 16.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=1c4be779a8993829c11895bd8b5d47191fc0786a1f0d4fb29eaea6592fb92c6d
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


##Importing required libraries

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

##Create SparkSession

In [12]:
spark = SparkSession.builder.getOrCreate()

##Create DataFrame

In [13]:
emp = [(1, "AAA", "dept1", 1000),
    (2, "BBB", "dept1", 1100),
    (3, "CCC", "dept1", 3000),
    (4, "DDD", "dept1", 1500),
    (5, "EEE", "dept2", 8000),
    (6, "FFF", "dept2", 7200),
    (7, "GGG", "dept3", 7100),
    (8, "HHH", "dept3", 3700),
    (9, "III", "dept3", 4500),
    (10, "JJJ", "dept5", 3400)]

dept = [("dept1", "Department - 1"),
        ("dept2", "Department - 2"),
        ("dept3", "Department - 3"),
        ("dept4", "Department - 4")

       ]

df = spark.createDataFrame(emp, ["id", "name", "dept", "salary"])
deptdf = spark.createDataFrame(dept, ["id", "name"])

##Display DataFrame

In [14]:
df.show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+



##Basic operations on DataFrames

###count

Count the number of rows

In [15]:
df.count()

10

###columns

Access the names of columns in the DataFrame

In [16]:
df.columns

['id', 'name', 'dept', 'salary']

###dtypes

Access the DataType of columns within the DataFrame

In [17]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('dept', 'string'),
 ('salary', 'bigint')]

###schema

Check how Spark stores the schema of the DataFrame

In [18]:
df.schema

StructType(List(StructField(id,LongType,true),StructField(name,StringType,true),StructField(dept,StringType,true),StructField(salary,LongType,true)))

###printSchema

In [19]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)



###select

Select particular columns from the DataFrame

In [20]:
df.select("id", "name").show()

+---+----+
| id|name|
+---+----+
|  1| AAA|
|  2| BBB|
|  3| CCC|
|  4| DDD|
|  5| EEE|
|  6| FFF|
|  7| GGG|
|  8| HHH|
|  9| III|
| 10| JJJ|
+---+----+



###filter

* Filter the rows based on some condition.

* Let's try to find the rows with id = 1.

* There are different ways to specify the condition.

In [21]:
# Execute below statement by uncommeting them.

df.filter(df["id"] == 1).show()
#df.filter(df.id == 1).show()
#df.filter(col("id") == 1).show()
#df.filter("id = 1").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



###drop

Drop a particular Column

In [22]:
newdf = df.drop("id")
newdf.show(2)

+----+-----+------+
|name| dept|salary|
+----+-----+------+
| AAA|dept1|  1000|
| BBB|dept1|  1100|
+----+-----+------+
only showing top 2 rows



###Aggregations

We can use the groupBy function to group the data and then use the "agg" function to perform aggregation on grouped data.

In [23]:
(df.groupBy("dept")
    .agg(
        count("salary").alias("count"),
        sum("salary").alias("sum"),
        max("salary").alias("max"),
        min("salary").alias("min"),
        avg("salary").alias("avg")
        ).show()
)

+-----+-----+-----+----+----+------+
| dept|count|  sum| max| min|   avg|
+-----+-----+-----+----+----+------+
|dept5|    1| 3400|3400|3400|3400.0|
|dept3|    3|15300|7100|3700|5100.0|
|dept1|    4| 6600|3000|1000|1650.0|
|dept2|    2|15200|8000|7200|7600.0|
+-----+-----+-----+----+----+------+



###Sorting

Sort the data based on "salary". By default, sorting will be done in Ascending order.

In [24]:
df.sort("salary").show(5)

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  4| DDD|dept1|  1500|
|  3| CCC|dept1|  3000|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+
only showing top 5 rows



In [25]:
# Sort the data in descending order.
df.sort(desc("salary")).show(5)

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  9| III|dept3|  4500|
|  8| HHH|dept3|  3700|
+---+----+-----+------+
only showing top 5 rows



###Derived Columns

We can use the "withColumn" function to derive the column based on existing columns…

In [26]:
df.withColumn("bonus", col("salary") * .1).show()

+---+----+-----+------+-----+
| id|name| dept|salary|bonus|
+---+----+-----+------+-----+
|  1| AAA|dept1|  1000|100.0|
|  2| BBB|dept1|  1100|110.0|
|  3| CCC|dept1|  3000|300.0|
|  4| DDD|dept1|  1500|150.0|
|  5| EEE|dept2|  8000|800.0|
|  6| FFF|dept2|  7200|720.0|
|  7| GGG|dept3|  7100|710.0|
|  8| HHH|dept3|  3700|370.0|
|  9| III|dept3|  4500|450.0|
| 10| JJJ|dept5|  3400|340.0|
+---+----+-----+------+-----+



###Joins

perform various types of joins on multiple DataFrames.

In [27]:
# Inner JOIN.
df.join(deptdf, df["dept"] == deptdf["id"]).show()

+---+----+-----+------+-----+--------------+
| id|name| dept|salary|   id|          name|
+---+----+-----+------+-----+--------------+
|  7| GGG|dept3|  7100|dept3|Department - 3|
|  8| HHH|dept3|  3700|dept3|Department - 3|
|  9| III|dept3|  4500|dept3|Department - 3|
|  1| AAA|dept1|  1000|dept1|Department - 1|
|  2| BBB|dept1|  1100|dept1|Department - 1|
|  3| CCC|dept1|  3000|dept1|Department - 1|
|  4| DDD|dept1|  1500|dept1|Department - 1|
|  5| EEE|dept2|  8000|dept2|Department - 2|
|  6| FFF|dept2|  7200|dept2|Department - 2|
+---+----+-----+------+-----+--------------+



###Left Outer Join

In [28]:
df.join(deptdf, df["dept"] == deptdf["id"], "left_outer").show()

+---+----+-----+------+-----+--------------+
| id|name| dept|salary|   id|          name|
+---+----+-----+------+-----+--------------+
| 10| JJJ|dept5|  3400| null|          null|
|  7| GGG|dept3|  7100|dept3|Department - 3|
|  8| HHH|dept3|  3700|dept3|Department - 3|
|  9| III|dept3|  4500|dept3|Department - 3|
|  1| AAA|dept1|  1000|dept1|Department - 1|
|  2| BBB|dept1|  1100|dept1|Department - 1|
|  3| CCC|dept1|  3000|dept1|Department - 1|
|  4| DDD|dept1|  1500|dept1|Department - 1|
|  5| EEE|dept2|  8000|dept2|Department - 2|
|  6| FFF|dept2|  7200|dept2|Department - 2|
+---+----+-----+------+-----+--------------+



###Right Outer Join

In [29]:
df.join(deptdf, df["dept"] == deptdf["id"], "right_outer").show()

+----+----+-----+------+-----+--------------+
|  id|name| dept|salary|   id|          name|
+----+----+-----+------+-----+--------------+
|   7| GGG|dept3|  7100|dept3|Department - 3|
|   8| HHH|dept3|  3700|dept3|Department - 3|
|   9| III|dept3|  4500|dept3|Department - 3|
|   1| AAA|dept1|  1000|dept1|Department - 1|
|   2| BBB|dept1|  1100|dept1|Department - 1|
|   3| CCC|dept1|  3000|dept1|Department - 1|
|   4| DDD|dept1|  1500|dept1|Department - 1|
|null|null| null|  null|dept4|Department - 4|
|   5| EEE|dept2|  8000|dept2|Department - 2|
|   6| FFF|dept2|  7200|dept2|Department - 2|
+----+----+-----+------+-----+--------------+



###Full Outer Join

In [30]:
df.join(deptdf, df["dept"] == deptdf["id"], "outer").show()

+----+----+-----+------+-----+--------------+
|  id|name| dept|salary|   id|          name|
+----+----+-----+------+-----+--------------+
|  10| JJJ|dept5|  3400| null|          null|
|   7| GGG|dept3|  7100|dept3|Department - 3|
|   8| HHH|dept3|  3700|dept3|Department - 3|
|   9| III|dept3|  4500|dept3|Department - 3|
|   1| AAA|dept1|  1000|dept1|Department - 1|
|   2| BBB|dept1|  1100|dept1|Department - 1|
|   3| CCC|dept1|  3000|dept1|Department - 1|
|   4| DDD|dept1|  1500|dept1|Department - 1|
|null|null| null|  null|dept4|Department - 4|
|   5| EEE|dept2|  8000|dept2|Department - 2|
|   6| FFF|dept2|  7200|dept2|Department - 2|
+----+----+-----+------+-----+--------------+



###SQL Queries

* Executing SQL like queries.

* We can perform data analysis by writing SQL like queries as well. In order to perform the SQL like queries, we need to register the DataFrame as a Temporary View.

In [31]:
# Register DataFrame as Temporary Table
df.createOrReplaceTempView("temp_table")

# Execute SQL-Like query.
spark.sql("select * from temp_table where id = 1").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



###Reading HIVE table as DataFrame

In [37]:
# DB_NAME : Name of the the HIVE Database
# TBL_NAME : Name of the HIVE Table

# Uncomment below statement and provide your Hive DB and Table details.
#df = spark.table("DB_NAME"."TBL_NAME")

###Save DataFrame as HIVE Table

In [38]:
# Uncomment below statement and provide your Hive DB and Table details.
#df.write.saveAsTable("DB_NAME.TBL_NAME")

# We can also select the "mode" argument for "overwrite", "append", "error" etc.
# df.write.saveAsTable("DB_NAME.TBL_NAME", mode="overwrite")

# Note: By default, the operation will save the DataFrame as a HIVE Managed/Internal table

###Saving the DataFrame as a HIVE External table

In [39]:
# Uncomment below statement and provide your Hive DB and Table details.
# Also, provide the path where you would like to save the Data.

# df.write.saveAsTable("DB_NAME.TBL_NAME", path=<location_of_external_table>)

###Create a DataFrame from CSV file

We can create a DataFrame using a CSV file and can specify various options like a separator, header, schema, inferSchema, and various other options.

In [40]:
# Un-comment below code and provide necessary details.
# df = spark.read.csv("path_to_csv_file", sep="|", header=True, inferSchema=True)

###Save a DataFrame as a CSV file

In [41]:
# Un-comment below code and provide necessary details.
# df.write.csv("path_to_CSV_File", sep="|", header=True, mode="overwrite")

###Create a DataFrame from a relational table

We can read the data from relational databases using a JDBC URL.

In [ ]:
# url : a JDBC URL of the form jdbc:subprotocol:subname
# TBL_NAME : Name of the relational table.
# USER_NAME : user name to connect to DataBase.
# PASSWORD: password to connect to DataBase.

# Un-comment below code and provide necessary details.
# relational_df = spark.read.format('jdbc')
#                        .options(url=url, dbtable= <TBL_NAME>, user= <USER_NAME>, password = <PASSWORD>)
#                        .load()

###Save the DataFrame as a relational table

We can save the DataFrame as a relational table using a JDBC URL.

In [42]:
# url : a JDBC URL of the form jdbc:subprotocol:subname
# TBL_NAME : Name of the relational table.
# USER_NAME : user name to connect to DataBase.
# PASSWORD: password to connect to DataBase.

# Un-comment below code and provide necessary details.
# relational_df.write.format('jdbc')
#                    .options(url=url, dbtable= <TBL_NAME>, user= <USER_NAME>, password = <PASSWORD>)
#                    .mode('overwrite')
#                    .save()

--------------------------------------------------------------------------------------------------------------------------------------
##**Part 2 Ends**
--------------------------------------------------------------------------------------------------------------------------------------